<a href="https://colab.research.google.com/github/JuanJoMontilla/Senales-y-Sistemas/blob/main/Dashboards_Streamlit_Ngrok_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Guía Streamlit en Colab y Ngrok - Dashboards

Elaborado por: Juan David Muñoz Buritica jmunozbu@unal.edu.co (Monitoria SyS 2024-1)
Revisado: Andrés Marino Álvarez Meza amalvarezme@unal.edu.co

- [Streamlit](https://streamlit.io/) es una librería destinada a crear y compartir aplicaciones o dashboards.
- Su objetivo es ser una herramienta fácil de usar que permita ejecutar scripts directamente y desplegarlos en un aplicativo web.
- Sin embargo, antes de ver su funcionamiento, debemos tener en cuenta que, si bien Streamlit permite correr el aplicativo, debemos alojarlo y "hacerlo visible" a la red.
- Existen diferentes métodos que cumplen dicho objetivo, también dependiendo del nivel de madurez que se requiera, bien sea simplemente testeo o producción. - Una alternativa eficaz, rápida y que puede ser fácilmente escalable es [Ngrok](https://ngrok.com/).

- Ngrok es un servicio y aplicativo globalmente distribuido que asegura, acelera y protege las aplicaciones y servicios de red.
- En este caso, nos servirá para crear el tunel http que nos deje visualizar el dashboard corriendo con Streamlit desde el entorno del cuadernillo o script de Python.
- El servicio gratuito de Ngrok, tras crear la cuenta tiene la posibilidad de mantener un agente activo simultáneo, cuya configuración está dada por el token de autenticación único por usuario y túnel.

**Nota: Cree una cuenta en Ngrok https://ngrok.com/ con su correo UNAL y guarde el token personal asignado**

# Ejemplo Dashboard con Streamlit y Ngrok

- Manipulación de audios desde Youtube

In [1]:
!pip install streamlit -q #instalación de librerías
!pip install pyngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 35.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 2.2 MB/s eta 0:00:00


In [2]:
!python3 -m pip install --force-reinstall https://github.com/yt-dlp/yt-dlp/archive/master.tar.gz
!pip install soundfile #librerias descarga Youtube y manejo de audios en python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 14.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached certifi-2024.7.4-py3-none-any.whl.metadata (2.2 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached charset_normalizer-3.3.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (33 kB)
  Using cached idna-3.8-py3-none-any.whl.metadata (9.9 kB)
Using cached requests-2.32.3-py3-none-any.whl (64 kB)
Using cached certifi-2024.7.4-py3-none-any.whl (162 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.4/121.4 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.3/157.3 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 41.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.

La estructura básica para correr un programa o script en streamlit es la siguiente.

In [3]:
"""
%%writefile app.py
import streamlit as st

st.write('# Hello World ')
st.write('## Run Streamlit on Colab with `pyngrok` ')
"""

"\n%%writefile app.py\nimport streamlit as st\n\nst.write('# Hello World ')\nst.write('## Run Streamlit on Colab with `pyngrok` ')\n"

- A continuación se muestra una app básica con algunas de las funcionalidades básicas para visualizar en dashboard.

- Debemos crear un archivo .py con los códigos del dashboard para su posterior visualización en Ngrok.

In [20]:
%%writefile detector.py

import streamlit as st
import pandas as pd
import numpy as np
import os
import subprocess
import soundfile as sf
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.manifold import TSNE
from pyngrok import ngrok
import yt_dlp as youtube_dl
import joblib


#widgets y funcionalidades de streamlit
st.title("YouTube Audio Analysis and Classification")
uploaded_file = st.file_uploader("Upload your Excel file", type=["xlsx"])

if uploaded_file:
    df = pd.read_excel(uploaded_file)
    st.write("Uploaded data:", df)

custom_url = st.text_input("Enter a new YouTube URL to process:", "")

song_duration = st.slider('Playtime for the song')

add_selectbox = st.sidebar.selectbox(
    'How many 5s pieces would you like to process?',
    ('1', '2', '3')
)

Overwriting detector.py


Como al momento de correr el script con Streamlit se requiere del archivo .py donde se encuentre el código, utilizamos %%writefile para crearlo. Posteriormente hacemos toda la configuración de Ngrok con el token y demás datos de protocolo.

In [21]:
token = '2lLFMBWiinIoZUi3bX7RF26eFeD_5KBTy8DqrMYoE2mf5Sqnc' #colocar aquí su token personal después de crear su cuenta con correo UNAL en Ngrok

In [22]:
from pyngrok import ngrok

# Set authentication token (unique per user)
ngrok.set_auth_token(token)

# Start Streamlit server on a specific port
!nohup streamlit run detector.py --server.port 5011 &

# Start ngrok tunnel to expose the Streamlit server
ngrok_tunnel = ngrok.connect(addr='5011', proto='http', bind_tls=True)

# Print the URL of the ngrok tunnel
print(' * Tunnel URL:', ngrok_tunnel.public_url)

nohup: appending output to 'nohup.out'
 * Tunnel URL: https://91c7-34-139-18-25.ngrok-free.app


In [24]:
#exit("Stopping the execution")

## Cómo usar Streamlit?



- Tan pronto como se ejecuta el script, un servidor local de Streamlit se inicia con el cuadernillo.
- La aplicación es como un lienzo, donde se pueden dibujar gráficos, texto, widgets, tablas, etc.
- Streamlit dispone de distintos métodos o funciones que permiten hacerlo, como por ejemplo, st.text() que escribe texto sin formato en la aplicación, y st.line_chart() que dibuja un gráfico de líneas.

La arquitectura de Streamlit permite escribir aplicaciones de la misma manera en que se escriben scripts en Python. Para esto, las aplicaciones de Streamlit tienen un flujo de datos único: cada vez que algo debe actualizarse en la pantalla, Streamlit vuelve a ejecutar todo el script de Python de principio a fin.

Esto puede ocurrir en dos situaciones:

- Cuando se modifica el código fuente de la aplicación.

- Cuando un usuario interactúa con los widgets en la aplicación. Por ejemplo, al arrastrar un deslizador, ingresar texto en un cuadro de entrada o hacer clic en un botón.

Veamos cómo funcionan algunas de estas opciones, tomando como base algunas funcionalidades del programa [youtube_detector](https://github.com/amalvarezme/SenalesSistemas/blob/master/3_SerieyTransformadaFourier/youtube_detector.ipynb).

In [25]:
import streamlit as st
import pandas as pd
import numpy as np
import os
import subprocess
import soundfile as sf
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.manifold import TSNE
from pyngrok import ngrok
import yt_dlp as youtube_dl
import joblib

Ponemos un título a la aplicación y página.

In [9]:
st.title("YouTube Audio Analysis and Classification")

2024-08-29 21:25:45.816 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-08-29 21:25:46.524 
  command:

    streamlit run /usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2024-08-29 21:25:46.536 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


DeltaGenerator()

Podemos dejar la opción para que el usuario suba un archivo, en este caso con el formato específico .xlsx

In [10]:
uploaded_file = st.file_uploader("Upload your Excel file", type=["xlsx"])

2024-08-29 21:25:46.590 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-08-29 21:25:46.595 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-08-29 21:25:46.602 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-08-29 21:25:46.612 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


Damos un mensaje de confirmación del archivo subido, y lo leemos con pandas.

In [11]:
if uploaded_file:
    df = pd.read_excel(uploaded_file)
    st.write("Uploaded data:", df)

Para el momento en el que queramos identificar una canción, podemos poner la opción de que el usuario introduza el link de esa canción.

In [12]:
custom_url = st.text_input("Enter a new YouTube URL to process:", "")

2024-08-29 21:25:46.669 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-08-29 21:25:46.671 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-08-29 21:25:46.672 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-08-29 21:25:46.674 Session state does not function when running a script without `streamlit run`
2024-08-29 21:25:46.675 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-08-29 21:25:46.677 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


### Procesamiento de Datos
- Ahora, una vez que se han introducido los datos, ocurre algo particular. La lista de canciones del archivo de excel van a utilizarse para entrenar un modelo sencillo que permita posteriormente hacer la clasificación.
- Existe una opción que nos permite guardar en caché elementos o partes (funciones) del script, que solo correrán una vez y no volverán a hacerlo a menos de que sea completamente necesario (han cambiado las condiciones o argumentos de la función).

Para evitar cálculos redundantes, Streamlit proporciona decoradores de caché como @st.cache_data y @st.cache_resource. Estos decoradores almacenan los resultados de cálculos costosos, los cuales pueden reutilizarse en diferentes ejecuciones de la aplicación.

- st.cache_data es la forma recomendada para almacenar en caché cálculos que devuelven datos o un objeto de datos serializable (por ejemplo, str, int, float, DataFrame, dict, list). Crea una nueva copia de los datos en cada llamada a la función, lo que lo hace seguro contra *mutaciones y condiciones de carrera*.

- st.cache_resource es la forma recomendada para almacenar en caché recursos globales como modelos de aprendizaje automático o conexiones a bases de datos. Usa st.cache_resource cuando tu función devuelve objetos no serializables que no quieres cargar múltiples veces. Devuelve el objeto en caché en sí, el cual se comparte entre todas las ejecuciones y sesiones sin copia ni duplicación. Si mutas un objeto que está en caché utilizando st.cache_resource, esa mutación existirá en todas las ejecuciones y sesiones.

Al agregar cálculos de larga duración a una aplicación, se puede usar st.progress() para mostrar el estado en tiempo real.

In [13]:
@st.cache_data
def download_ytvid_as_mp3(video_url,name):
    #video_url = input("enter url of youtube video:")
    video_info = youtube_dl.YoutubeDL().extract_info(url = video_url,download=False)
    filename = f"{name}.mp3"
    options={
        'format':'bestaudio/best',
        'keepvideo':False,
        'outtmpl':filename,
    }

    with youtube_dl.YoutubeDL(options) as ydl:
        ydl.download([video_info['webpage_url']])

    print("Download complete... {}".format(filename))

@st.cache_resource
def load_and_process_audio_files(path='results/'):
    # Function to train the model from the Excel file
    pass

2024-08-29 21:25:46.687 No runtime found, using MemoryCacheStorageManager


También existen otros elementos algo más interactivos (widgets) para el usuario que pueden cumplir funciones importantes, como st.button(), st.slider() o st.selectbox(). Estos se tratan como si fueran simplemente variables.

In [14]:
def process_audio_from_link(df):
  pass

if st.button("Download and Process Audio"):
    path = process_audio_from_link(df)
    x_t, label, name_c, fs = load_and_process_audio_files(path)

2024-08-29 21:25:46.700 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-08-29 21:25:46.702 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-08-29 21:25:46.704 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-08-29 21:25:46.705 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [15]:
song_duration = st.slider('Playtime for the song')

2024-08-29 21:25:46.712 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-08-29 21:25:46.714 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-08-29 21:25:46.715 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-08-29 21:25:46.717 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-08-29 21:25:46.719 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


### Visualización de Datos
Streamlit brinda cierta flexibilidad y opciones para visualizar datos, bien sea con funciones propias o su integración con pyplot.

In [16]:
"""
st.write("Time Domain Signals")
st.line_chart(x_t.mean(axis=-1)) # Gráfico en el tiempo
"""

'\nst.write("Time Domain Signals")\nst.line_chart(x_t.mean(axis=-1)) # Gráfico en el tiempo\n'

In [17]:
"""
plt.plot(vf, abs(Xw).T)
plt.xlabel('Frequency (Hz)')
plt.ylabel('|X(f)|')
st.pyplot(plt)
"""

"\nplt.plot(vf, abs(Xw).T)\nplt.xlabel('Frequency (Hz)')\nplt.ylabel('|X(f)|')\nst.pyplot(plt)\n"

Streamlit en ocasiones recomienda usar st.write() gracias a su funcionalidad "Magic". Podemos pasarle casi cualquier cosa a st.write() y automáticamente encontrará la mejor manera de representar esos datos (texto, datos, figuras, etc). Sin embargo, para un mayor control, pueden usarse los métodos mencionados antes.

### Presentación del dashboard

Streamlit facilita la organización de los widgets en un panel lateral izquierdo con st.sidebar(). Cada elemento que se pasa a st.sidebar() queda fijado a la izquierda, permitiendo a los usuarios concentrarse en el contenido de la aplicación mientras siguen teniendo acceso a los controles de la interfaz.

In [18]:
# Add a selectbox to the sidebar:
add_selectbox = st.sidebar.selectbox(
    'How would you like to be contacted?',
    ('Email', 'Home phone', 'Mobile phone')
)

2024-08-29 21:25:46.742 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-08-29 21:25:46.758 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-08-29 21:25:46.762 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-08-29 21:25:46.764 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-08-29 21:25:46.766 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


Más allá del panel lateral, Streamlit ofrece varias otras formas de controlar la disposición de tu aplicación. st.columns() permite colocar widgets uno al lado del otro, y st.expander() permite ahorrar espacio al ocultar contenido extenso.

A medida que las aplicaciones crecen, resulta útil organizarlas en múltiples páginas. Esto facilita la gestión de la aplicación y la navegación como usuario. Streamlit ofrece una forma sencilla de crear aplicaciones multipágina.
- En la carpeta donde está el script principal, se crea una nueva carpeta para las páginas [pages].
- Se añaden nuevos archivos .py en esta carpeta para añadir más páginas a la app.
- Se corre sl script principal de la misma manera.

Para mayor detalle en el uso de streamlit, podemos recurrir a su propia documentación, con ejemplos y otros detalles: https://docs.streamlit.io/get-started